# load

> Module for loading in new datasets and performing validation checks on them  

In [ ]:
#| default_exp load

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

import csv
from enum import Enum
from typing import List

from sqlalchemy import Boolean, Column, Integer, String, create_engine, text
from sqlalchemy.orm import declarative_base, sessionmaker

from enum import Enum, auto
from typing import Dict, List, Optional, Tuple, Union

from pathlib import Path


In [ ]:
#| export

Base = declarative_base()

class StudyRecord(Base):
    """
    Class modeling the records in our sqlite database.

    The attributes of this class must match the column names in NavySleepResearch/data_sets.csv. Using
    Python's built-in CSVReader class, we get an array of dicts that are easy to turn into StudyRecord
    objects via dict unpacking (** operator) and kwarg matching, which is a fancy way to say:
      new_record = StudyRecord(**csv_row_dict)
    """

    __tablename__ = "data_sets"

    id = Column(
        Integer, primary_key=True
    )  # database id column, not id of patient within study

    # Identification of patient and data set
    subject_id = Column(String)
    data_set_name = Column(String)

    # Patient metadata
    age = Column(Integer)
    sex = Column(Integer)  # Integer: maps to Enum
    race = Column(Integer)  # Integer: maps to Enum
    has_apnea = Column(Boolean)
    has_idiopathic_hypersomnia = Column(Boolean)
    has_sleep_paralysis = Column(Boolean)
    has_restless_leg_syndrome = Column(Boolean)
    has_insomnia = Column(Boolean)
    has_parasomnia = Column(Boolean)
    has_rem_sleep_disorder = Column(Boolean)
    has_narcolepsy = Column(Boolean)
    has_nocturia = Column(Boolean)

    # Identify what data is available and where it is
    psg_location = Column(String)
    accelerometer_location = Column(String)
    heartrate_location = Column(String)
    activity_location = Column(String)

    def __repr__(self) -> str:
        return f"{self.subject_id}:{self.data_set_name}"

    def diagnosis(self) -> str:
        known_conditions = [
            "apnea",
            "idiopathic_hypersomnia",
            "sleep_paralysis",
            "restless_leg_syndrome",
            "insomnia",
            "parasomnia",
            "rem_sleep_disorder",
            "narcolepsy",
            "nocturia",
        ]

        self_dict = (
            self.__dict__
        )  # lets us paste known_conditions into dict keys for easy iteration

        found_conditions = ""

        for condition in known_conditions:
            if self_dict[f"has_{condition}"]:
                found_conditions += f" -- {condition.replace('_', ' ')}\n"
        return found_conditions


class DatabaseLiaison:
    def __init__(self, db_filename: str):
        self.engine = create_engine(f"sqlite:///{db_filename}")
        self.Session_maker = sessionmaker(bind=self.engine)
        Base.metadata.create_all(
            bind=self.engine
        )  # makes the actual tables in database, if they don't exist

    def add(self, record: StudyRecord):
        session = self.Session_maker()
        session.add(record)
        session.commit()

    def add_all(self, records: List[StudyRecord]):
        session = self.Session_maker()
        session.add_all(records)
        session.commit()

    def select(self, selector: str = "") -> List[StudyRecord]:
        """
        Runs raw SQL on database; selector adds a WHERE clause to SELECT * FROM data_sets
        The default "" selects everything, equivalent to '''SELECT * FROM data_sets'''
        :param selector: WHERE clause contents
        :return: List of study records matching the selector.
        """
        session = self.Session_maker()
        return session.query(StudyRecord).filter(text(selector)).all()

    def to_csvDictWriterDicts(self) -> List[dict]:
        """
        Converts database to a list of dicts ready to be sent into a CSV by csv.DictWriter.
        :return: `[dict]` one per row with keys being the columns.
        """
        session = self.Session_maker()
        records = [r.__dict__ for r in session.query(StudyRecord).all()]

        for r in records:
            # pop those keys that are not part of CSV
            r.pop("id")  # db PK, dont persist this
            r.pop("_sa_instance_state")  # artifact of SQLAlchemy class.

        return records

    def to_csv(self, filename: str):
        records = self.to_csvDictWriterDicts()
        with open(filename, "w", newline="") as csvfile:
            # This must be hardcoded to ensure deterministic order, AFAIK
            fieldnames = [
                "subject_id",
                "data_set_name",
                "age",
                "sex",
                "race",
                "has_apnea",
                "has_idiopathic_hypersomnia",
                "has_sleep_paralysis",
                "has_restless_leg_syndrome",
                "has_insomnia",
                "has_parasomnia",
                "has_rem_sleep_disorder",
                "has_narcolepsy",
                "has_nocturia",
                "psg_location",
                "accelerometer_location",
                "heartrate_location",
                "activity_location",
            ]

            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for rec in records:
                writer.writerow(rec)


In [ ]:
class KnownFeatures(Enum):
    """
    Expresses to the system which features from data to include.
    This lives here for cyclic import reasons...
    """

    ACTIVITY = auto()
    ACCELEROMETER = auto()
    HEARTRATE = auto()
    PSG = auto()

    # Not settable, derived
    ACCEL_SPECTRO = auto()
    MO_SPECTRO = auto()  # Mads Olsen preprocessing

    @property
    def base_feature(self) -> "KnownFeatures":
        if self.is_accelerometer_based:
            return KnownFeatures.ACCELEROMETER
        elif self.is_activity_based:
            return KnownFeatures.ACTIVITY
        elif self.is_heartrate_based:
            return KnownFeatures.HEARTRATE
        else:
            return self

    @property
    def is_derived(self) -> bool:
        match self:
            case KnownFeatures.ACTIVITY | KnownFeatures.ACCELEROMETER | KnownFeatures.HEARTRATE | KnownFeatures.PSG:
                return False
            case _:
                return True

    @property
    def is_spectral(self) -> bool:
        return self in [KnownFeatures.ACCEL_SPECTRO, KnownFeatures.MO_SPECTRO]

    @property
    def is_accelerometer_based(self) -> bool:
        # Convenience + readability helper
        return self in [
            KnownFeatures.ACCELEROMETER,
            KnownFeatures.ACCEL_SPECTRO,
            KnownFeatures.MO_SPECTRO,
        ]

    @property
    def is_activity_based(self) -> bool:
        return self in [KnownFeatures.ACTIVITY]

    @property
    def is_heartrate_based(self) -> bool:
        return self in [KnownFeatures.HEARTRATE]


# Determines the names of folders inside a data set's root
# where we expect to find specific data CSVs.
ACCELEROMETER_FOLDER_NAME = "cleaned_accelerometer"
HEARTRATE_FOLDER_NAME = "cleaned_heartrate"
PSG_FOLDER_NAME = "cleaned_psg"
ACTIGRAPHY_FOLDER_NAME = "cleaned_activity"


# class DataRecord:
#     """
#     Class that understands how to take study database records and provide data to the pipeline.
#     """

#     def __init__(
#         self, study_record: StudyRecord, data_folder: Optional[str | Path] = None
#     ):
#         self.record = study_record

#         self.data_root = (
#             Path(__file__).parent.parent.parent.joinpath("data")
#             if data_folder is None
#             else Path(data_folder)
#         )
#         self.data_folder = self.data_root.joinpath(str(self.record.data_set_name))
#         # These will be populated by actual data that can be operated on by classifiers
#         self._data: Dict[KnownFeatures, TimeseriesRecording] = {
#             KnownFeatures.PSG: None,
#             KnownFeatures.ACCELEROMETER: None,
#             KnownFeatures.HEARTRATE: None,
#             KnownFeatures.ACTIVITY: None,
#         }

#     @property
#     def psg_data(self) -> Union[PSGStagingRecord, PSGSleepWakeRecord] | None:
#         return self._data[KnownFeatures.PSG]

#     @psg_data.setter
#     def psg_data(self, value):
#         assert (value is None) or issubclass(type(value), PSGRecord)
#         self._data[KnownFeatures.PSG] = value

#     @property
#     def acceleration_data(self) -> AccelerometerRecord | None:
#         return self._data[KnownFeatures.ACCELEROMETER]

#     @acceleration_data.setter
#     def acceleration_data(self, value):
#         assert (value is None) or isinstance(value, AccelerometerRecord)
#         self._data[KnownFeatures.ACCELEROMETER] = value

#     @property
#     def heartrate_data(self) -> HeartrateRecord | None:
#         return self._data[KnownFeatures.HEARTRATE]

#     @heartrate_data.setter
#     def heartrate_data(self, value):
#         assert (value is None) or isinstance(value, HeartrateRecord)
#         self._data[KnownFeatures.HEARTRATE] = value

#     @property
#     def activity_count_data(self) -> ActivityCountRecord | None:
#         return self._data[KnownFeatures.ACTIVITY]

#     @activity_count_data.setter
#     def activity_count_data(self, value):
#         assert (value is None) or isinstance(value, ActivityCountRecord)
#         self._data[KnownFeatures.ACTIVITY] = value

#     @property
#     def data_set_name(self):
#         return self.record.data_set_name

#     @property
#     def subject_id(self):
#         return self.record.subject_id

#     @property
#     def absolute_accelerometer_path(self) -> str:
#         return self.data_folder.joinpath(ACCELEROMETER_FOLDER_NAME).joinpath(
#             self.record.accelerometer_location
#         )

#     def max_time(self, over: List[KnownFeatures] = list(KnownFeatures)) -> float:
#         times = []
#         for feature in over:
#             feat = self._data.get(feature)
#             if feat is not None:
#                 times += [feat.time.max()]
#         # times is array of maximum time on each non-None feature
#         # min of maximum times gives maximum time intersection
#         return min(times) if times else np.inf

#     def min_time(self, over: List[KnownFeatures] = list(KnownFeatures)) -> float:
#         times = []
#         for feature in over:
#             feat = self._data.get(feature)
#             if feat is not None:
#                 times += [feat.time.min()]
#         # times is array of minimum time on each non-None feature
#         # max of minimum times gives minimum time intersection
#         return max(times) if times else -1.0 * np.inf

#     def trim(self, start: float, end: float):
#         """
#         Trims all of the timeseries loaded on this object to have .time values
#         between `start` and `end`, inclusive. (`start` <= `t` <= `end` for all times `t` in self)
#         """
#         start = floor(start)
#         end = ceil(end)
#         for k in self._data:
#             if self._data[k] is not None:
#                 self._data[k].trim(start, end)

#     def samples(self, feature: KnownFeatures) -> Optional[np.ndarray]:
#         """
#         Get inputs/data for samples in this record
#         :param feature: `KnownFeatures` element you want
#         :return: np.ndarray of shape (n_samples,), or (n_samples, 3) for accel, or `None` if this data not loaded.
#         """
#         data_obj = self._data.get(feature.base_feature)
#         if data_obj is None:
#             return None

#         if feature.is_activity_based:
#             return data_obj.count
#         elif feature.is_heartrate_based:
#             return data_obj.heartrate
#         elif feature.is_accelerometer_based:
#             data_obj: AccelerometerRecord
#             return data_obj.xyz

#     def times(self, feature: KnownFeatures) -> Optional[np.ndarray]:
#         """
#         Get time axis array for the given feature on this record.
#         """

#         feature_data = self._feature_select(feature)
#         if feature_data is None:
#             return None
#         return feature_data.time

#     def _feature_select(self, feature: KnownFeatures) -> Optional[FeatureRecord]:
#         if feature.is_activity_based:
#             return self.activity_count_data
#         elif feature.is_heartrate_based:
#             return self.heartrate_data
#         elif feature.is_accelerometer_based:
#             return self.acceleration_data
#         else:
#             return None

#     def load_data(self, load_accel: bool = False, verbose: bool = False):
#         """
#         Mutating function. Loads data from locations we have. Empty CSV cells are loaded as '' which is Falsey.
#         :param load_accel: `bool` controlling if we load accelerometer data or not. This is often huge, might crash your computer.
#         :return: `None` since this is a self-mutation method
#         """

#         # PSG
#         start = time.time()
#         if verbose:
#             print("Loading PSG...")
#         worked = True
#         try:
#             self._load_psg()
#         except Exception as e:
#             if verbose:
#                 print(e)
#                 print("didnt load", self.record.psg_location)
#             worked = False
#         if verbose and worked:
#             print(f"PSG loaded, took {time.time() - start:0.2f} seconds")
#         # Heart Rate
#         start = time.time()
#         if verbose:
#             print("Loading Heart rate...")
#         worked = True
#         try:
#             self._load_heartrate()
#         except Exception as e:
#             if verbose:
#                 print(e)
#                 print("didnt load", self.record.heartrate_location)
#             worked = False
#         if verbose and worked:
#             print(f"Heartrate loaded, took {time.time() - start:0.2f} seconds")

#         # Actigraphy
#         start = time.time()
#         if verbose:
#             print("Loading activity counts...")
#         worked = True
#         try:
#             self._load_activity()
#         except Exception as e:
#             if verbose:
#                 print(e)
#                 print("didnt load", self.record.activity_location)
#             worked = False
#         if verbose and worked:
#             print(f"Activity counts loaded, took {time.time() - start:0.2f} seconds")

#         # High-resolution Accelerometer
#         if load_accel:
#             start = time.time()
#             if verbose:
#                 print("Loading Accelerometer...")
#             try:
#                 self._load_accel()
#                 worked = True
#             except Exception as e:
#                 if verbose:
#                     print(type(e))
#                     print(e)
#                     print("didnt load", self.record.accelerometer_location)
#                 worked = False
#             if verbose and worked:
#                 print(f"Accelerometer loaded, took {time.time() - start:0.2f} seconds")

#     def _load_accel(self):
#         """
#         Store logic for how to load a triaxial accelerometer file here!
#         :return: AccelerometerRecord with data loaded as appropriate
#         """
#         specific_path = self.data_folder.joinpath(ACCELEROMETER_FOLDER_NAME)
#         if not self.record.accelerometer_location:
#             # Attempt to infer based on specific path
#             for file in os.listdir(specific_path):
#                 if self.subject_id in file:
#                     self.record.accelerometer_location = file
#                     break
#         location = specific_path.joinpath(str(self.record.accelerometer_location))
#         # print(location)
#         header_count, delim = determine_header_rows_and_delimiter(location)
#         if header_count is None or delim is None:
#             raise ValueError(
#                 "The location provided could not be parsed as CSV"
#                 + f"\npath: {location.absolute()}"
#             )

#         array_data = np.loadtxt(location, skiprows=header_count, delimiter=delim)
#         self.acceleration_data = AccelerometerRecord(
#             time=array_data[:, 0],
#             _xyz=array_data[:, 1:],
#         )

#         return self.acceleration_data

#     def _load_heartrate(self):
#         """
#         Store logic for how to load a heart rate file here!
#         :return: HeartrateRecord with data loaded as appropriate
#         """
#         specific_path = self.data_folder.joinpath(HEARTRATE_FOLDER_NAME)
#         if not self.record.heartrate_location:
#             # Attempt to infer based on specific path
#             for file in os.listdir(specific_path):
#                 if self.subject_id in file:
#                     self.record.heartrate_location = file
#                     break
#         location = specific_path.joinpath(str(self.record.heartrate_location))
#         header_count, delim = determine_header_rows_and_delimiter(location)
#         if header_count is None or delim is None:
#             raise ValueError(
#                 "The location provided could not be parsed as CSV"
#                 + f"\npath: {location.absolute()}"
#             )

#         array_data = np.loadtxt(location, skiprows=header_count, delimiter=delim)
#         self.heartrate_data = HeartrateRecord(
#             time=array_data[:, 0], heartrate=array_data[:, 1]
#         )

#         return self.heartrate_data

#     def _load_psg(self):
#         """
#         Store logic for how to load a PSG 4 or 5 stage file here!
#         :return: PSGStagingRecord with data loaded as appropriate
#         """
#         specific_path = Path(self.data_folder).joinpath(PSG_FOLDER_NAME)
#         if not self.record.psg_location:
#             # Attempt to infer based on specific path
#             files = os.listdir(specific_path)
#             for file in files:
#                 if self.subject_id in file:
#                     self.record.psg_location = file
#                     break

#         location = specific_path.joinpath(str(self.record.psg_location))
#         header_count, delim = determine_header_rows_and_delimiter(location)
#         if header_count is None or delim is None:
#             raise ValueError(
#                 "The location provided could not be parsed as CSV"
#                 + f"\npath: {location.absolute()}"
#             )

#         array_data = np.loadtxt(location, skiprows=header_count, delimiter=delim)

#         self.psg_data = PSGStagingRecord(
#             time=array_data[:, 0],
#             # Safer to assume SleepStages, since W=0/S=1 still loads with .value > 0 meaning sleep,
#             # so when we call .to_sleep_wake the correct labels will be decided.
#             psg=[
#                 SleepStages(int(d)) for d in array_data[:, 1]
#             ],  # int(d) to safeguard against eg 1 vs 1.0
#         )

#         return self.psg_data

#     def _load_activity(self):
#         """
#         Store logic for how to load an activity count file here!
#         :return: AccelerometerRecord with data loaded as appropriate
#         """
#         specific_path = self.data_folder.joinpath(ACTIGRAPHY_FOLDER_NAME)
#         if not self.record.activity_location:
#             files = os.listdir(specific_path)
#             # Attempt to infer based on specific path
#             for file in files:
#                 if self.subject_id in file:
#                     self.record.activity_location = file
#                     break
#         location = specific_path.joinpath(str(self.record.activity_location))
#         header_count, delim = determine_header_rows_and_delimiter(location)
#         if header_count is None or delim is None:
#             raise ValueError(
#                 "The location provided could not be parsed as CSV"
#                 + f"\npath: {location.absolute()}"
#             )

#         array_data = np.loadtxt(location, skiprows=header_count, delimiter=delim)
#         self.activity_count_data = ActivityCountRecord(
#             time=array_data[:, 0], count=array_data[:, 1]
#         )

#         return self.activity_count_data

#     def fill_gaps(self, in_: List[KnownFeatures] = []):
#         if len(self.psg_data.time) < 2:
#             return
#         psg_dt = self.psg_data.time[1] - self.psg_data.time[0]
#         self.psg_data.fill_gaps(max_gap_seconds=psg_dt, fill_dt=psg_dt)

#         in_ = [i.base_feature for i in in_]

#         for feature in in_:
#             if feature == KnownFeatures.PSG:
#                 continue  # already done
#             f_data = self._data.get(feature)
#             if f_data is None:
#                 continue
#             for j, t in enumerate(self.psg_data.time):
#                 # We dont just want to look for gaps,
#                 # we want to find gaps in f_data that contain an entire PSG epoch
#                 t_plus = f_data.time[f_data.time >= t]
#                 has_gap_at_t = (len(t_plus) == 0) or ((t_plus[0] - t) > psg_dt)

#                 if has_gap_at_t:
#                     psg_type = type(self.psg_data.psg[j])
#                     self.psg_data.psg[j] = psg_type.UNSCORED
#             self._data[feature].fill_gaps(
#                 max_gap_seconds=psg_dt, fill_dt=np.diff(f_data.time).mean()
#             )

#         for feature in self._data:
#             # Any other data should be discarded, it is now totally out of sync with transformations
#             if feature in [*in_, KnownFeatures.PSG]:
#                 continue
#             self._data[feature] = None

#     def tidy(
#         self,
#         verbose: bool = False,
#         acc_Hz: int = 50,
#         mask_gaps_in: List[KnownFeatures] = [],
#     ):
#         # Align and aggregate activity counts, PSG, heartrate
#         mask_gaps_in = [m.base_feature for m in mask_gaps_in]

#         mask_gaps_in = list(set([KnownFeatures.PSG] + mask_gaps_in))

#         if self.psg_data is None:
#             return
#         # All data should be conformed to PSG
#         start_time = min(self.psg_data.time)
#         end_time = max(self.psg_data.time)

#         self.trim(start_time, end_time)

#         if verbose:
#             print("start_time", start_time)
#             print("end_time", end_time)
#             print((end_time - start_time) // 3600, "hours")

#         # Identify gaps in model input data, match to epochs of PSG and sub -1
#         self.fill_gaps(in_=mask_gaps_in)

#         self.bin_to_psg()
#         if self._data.get(KnownFeatures.ACCELEROMETER) is not None:
#             self._data[KnownFeatures.ACCELEROMETER].resample(Hz=acc_Hz)

#     def bin_to_psg(self):
#         if self.psg_data is None:
#             raise ValueError("No psg data found!")
#         for f in self._data:
#             f_data = self._data.get(f)
#             if f_data is None:
#                 continue
#             self._data[f].bin_based_on(self.psg_data.time)

#     def __str__(self):
#         return str(self.record)

#     def __getstate__(self) -> dict:
#         # Returns lightened version of self. Sets all the data to None, retains loading info.
#         return DataRecord(study_record=self.record).__dict__


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# from pisces.records import PSGStagingRecord

In [ ]:
import pisces
import pkgutil

In [ ]:
#  [name for _, name, _ in pkgutil.iter_modules(['pisces'])]

In [ ]:
help(pisces)

Help on package pisces:

NAME
    pisces

PACKAGE CONTENTS
    _modidx
    coooooore
    core
    load

VERSION
    0.0.1

FILE
    /Users/ojwalch/Documents/pisces/pisces/__init__.py


